### Setting up a Demo Foundry Agent via Code

### Installing Required Libraries

In [1]:
%pip install azure-ai-projects==2.0.0b2
%pip install openai==1.109.1
%pip install python-dotenv
%pip install azure-identity

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Setting up the Environment Variables

In [2]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv(dotenv_path=os.path.join(os.pardir, ".env"))

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Setting up the Foundry Project Client

In [3]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Setting up our Batman Agent

In [4]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "it-support-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are an IT support agent. Respond to all queries with helpful and accurate IT support information.",
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

Agent created (id: it-support-agent:1, name: it-support-agent, version: 1)


### Creating a Conversation Object for Agent Chat System

In [5]:
# creating an openai client first
openai_client = project_client.get_openai_client()

# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

Created conversation with id: conv_c976db71634e2a8b00M6YgMSNN53zkthjRM6CcAe181BkhgkT2


### Chat with the Agent

In [6]:
chat = True

while chat:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        chat = False
        print("Exiting chat. Goodbye!")
    else:
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )
        print(f"user: {user_input}")
        print(f"It-Support: {response.output_text}")

user: my internet not working 
It-Support: I'm here to help! Let's try to get your internet working again. Please follow these steps:

1. **Check Cables and Connections**:  
   - Ensure all cables (power, Ethernet, phone line, etc.) are securely plugged in to your modem/router and your device.

2. **Restart Your Modem and Router**:  
   - Unplug the power from both your modem and router.  
   - Wait for 30 seconds, then plug them back in.  
   - Wait 2-3 minutes for all lights to stabilize.

3. **Restart Your Device**:  
   - Restart your computer, phone, or other device.

4. **Check for Internet Light**:  
   - On your modem/router, look for the "Internet" or "WAN" light.  
   - Is it solid, blinking, or off? (Let me know.)

5. **Test Other Devices**:  
   - Try connecting to the internet with a different device (phone, tablet, laptop).

6. **Check Wi-Fi Settings**:  
   - Make sure you’re connected to the correct Wi-Fi network.  
   - Try forgetting the network and reconnecting with 